<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [1]:
#!wget https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

--2020-09-02 20:50:16--  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip
Resolving s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)... 67.228.254.196
Connecting to s3-api.us-geo.objectstorage.softlayer.net (s3-api.us-geo.objectstorage.softlayer.net)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 261483817 (249M) [application/zip]
Saving to: ‘concrete_data_week4.zip’

concrete_data_week4 100%[===================>] 249.37M  25.5MB/s    in 8.4s    

2020-09-02 20:50:26 (29.8 MB/s) - ‘concrete_data_week4.zip’ saved [261483817/261483817]



In [ ]:
#!unzip concrete_data_week4.zip

Archive:  concrete_data_week4.zip
   creating: concrete_data_week4/
   creating: concrete_data_week4/valid/
   creating: concrete_data_week4/valid/positive/
  inflating: concrete_data_week4/valid/positive/16679_1.jpg  
   creating: __MACOSX/
   creating: __MACOSX/concrete_data_week4/
   creating: __MACOSX/concrete_data_week4/valid/
   creating: __MACOSX/concrete_data_week4/valid/positive/
  inflating: __MACOSX/concrete_data_week4/valid/positive/._16679_1.jpg  
  inflating: concrete_data_week4/valid/positive/19463.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19463.jpg  
  inflating: concrete_data_week4/valid/positive/19041_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19041_1.jpg  
  inflating: concrete_data_week4/valid/positive/19004_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._19004_1.jpg  
  inflating: concrete_data_week4/valid/positive/18041_1.jpg  
  inflating: __MACOSX/concrete_data_week4/valid/positive/._18041_1.jpg  

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)

In [3]:
training_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size = (224, 224),
    batch_size = 100,
    class_mode='categorical'
)

Found 30000 images belonging to 2 classes.


In [4]:
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size = (224, 224),
    batch_size = 100,
    class_mode = 'categorical'    
)

Found 9500 images belonging to 2 classes.


In [5]:
vgg_model = Sequential()

In [7]:
vgg_model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet'
))

Exception: URL fetch failure on https://storage.googleapis.com/tensorflow/keras-applications/vgg16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5: None -- [Errno -3] Temporary failure in name resolution

In [7]:
vgg_model.add(Dense(2 , activation="softmax"))

In [8]:
vgg_model.layers[0].trainable = False

In [9]:
vgg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense (Dense)                (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
vgg_model.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics = ['mse' , 'accuracy']
)

In [12]:
steps_per_epoch_training = len(training_generator)
steps_per_epoch_validation = len(validation_generator)

fit_history = vgg_model.fit(
    training_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=2,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 300 steps, validate for 95 steps
Epoch 1/2
300/300 [==============================] - 75s 248ms/step - loss: 0.0642 - mse: 0.0157 - accuracy: 0.9805 - val_loss: 0.0182 - val_mse: 0.0035 - val_accuracy: 0.9959
Epoch 2/2
300/300 [==============================] - 319s 1s/step - loss: 0.0146 - mse: 0.0028 - accuracy: 0.9966 - val_loss: 0.0111 - val_mse: 0.0023 - val_accuracy: 0.9975


<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [16]:
resnet_model = load_model('classifier_resnet_model.h5')

In [21]:
test_generator = data_generator.flow_from_directory (
    'concrete_data_week4/test',
    target_size = (224 , 224),
    shuffle = False,
    class_mode = 'categorical'
)

Found 500 images belonging to 2 classes.


`evaluatte_generator` according to the documentation for tensorflow 2.3

<aside class="warning">
Evaluates the model on a data generator. (deprecated)
<strong>Warning:</strong>
    <p>THIS FUNCTION IS DEPRECATED. It will be removed in a future version.</p>
    <p>Please use Model.evaluate, which supports generators.</p>

</aside>    

In [26]:
#note, evaluate_generator has been deprecated in 2.x release
test_scores_resnet = resnet_model.evaluate(test_generator)

  ...
    to  
  ['...']
16/16 [==============================] - 1s 50ms/step - loss: 0.1131 - accuracy: 0.9620 - mse: 0.0317


In [27]:
print('The ResNet50 had an accuracy of ' , test_scores_resnet[1])

The ResNet50 had an accuracy of  0.962


In [28]:
#note, evaluate_generator has been deprecated in 2.x release
test_scores_vgg = vgg_model.evaluate(test_generator)

  ...
    to  
  ['...']
16/16 [==============================] - 1s 61ms/step - loss: 0.0094 - mse: 0.0016 - accuracy: 0.9980


In [25]:
print('The Vgg16 had an accuracy of ', test_scores_vgg[2])

The Vgg16 had an accuracy of  0.998


<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [29]:
#note predict_generator is deprecated in tensor flow 2.x
vgg_predictions = vgg_model.predict(test_generator)

In [39]:
vgg_predictions[:5]

array([[0.99750704, 0.00249299],
       [0.9600755 , 0.03992452],
       [0.9718208 , 0.02817926],
       [0.992592  , 0.00740808],
       [0.99192494, 0.00807504]], dtype=float32)

In [40]:
test_generator.class_indices

{'negative': 0, 'positive': 1}

In [32]:
resnet_predictions =  resnet_model.predict(test_generator)

In [33]:
resnet_predictions[:5]

array([[9.9997222e-01, 2.7803757e-05],
       [9.9999464e-01, 5.3268350e-06],
       [9.9999511e-01, 4.9187925e-06],
       [9.9997735e-01, 2.2630315e-05],
       [9.9964058e-01, 3.5944665e-04]], dtype=float32)

In [42]:
print('The predictions of the first 5 images are Negative, Negative, Negative, Negative, Negative' )

The predictions of the first 5 images are Negative, Negative, Negative, Negative, Negative


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).